### Nonlinear pendulum
This notebook illustrates the dynamics of a nonlinear pendulum by connecting the phase portrait of a nonlinear model of a pendulum with the motion of a physical pendulum. The next cell contains the code for solving and visualizing solutions of the pendulum model.  

In [3]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.backend_bases import MouseButton
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve

toggle = False # if false, solve ODE; if true, find equilibria
labels = [0, 1]
    
def generate_phase_portrait(f, pars, Nframes=200, yrange=[-3.5, 3.5]):
    xrange = [-4, 4]
    Neval = 10
    T = 0.1*Nframes
    
    fig, ax = plt.subplots(1,2,figsize=(8,4))
    plt.subplots_adjust(bottom=0.2)

    button = plt.axes([0.15, 0.0, 0.26, 0.05])   # [left, bottom, width, height]
    labels[0] = button.text(0.05, 0.3, "Click here to find equilibria", fontsize=10)
    labels[1] = button.text(0.10, 0.3, "Click here to solve ODE", fontsize=10)
    labels[0].set_visible(True)
    labels[1].set_visible(False)
    button.set(xticks=[], yticks=[])
    button.set_facecolor('tab:green')

    ax[0].set_title("Phase portrait")
    ax[0].set_xlabel('angle')
    ax[0].set_ylabel('angular velocity')
    ax[0].set_xlim(xrange)
    ax[0].set_ylim(yrange)
    ax[0].grid(True)
    line1, = ax[0].plot([], [], linewidth=2, color="tab:purple", zorder=2)
    line2, = ax[0].plot([], [], 'o', markersize=7.5, color="tab:purple", zorder=2)
    plotdf(f, pars, xrange, yrange, [11, 11], ax[0])
    
    ax[1].set_title("Pendulum")
    ax[1].axis([-1.2, 1.2, -1.2, 1.2])
    ax[1].set_aspect('equal')
    line3, = ax[1].plot([], [], linewidth=4, color="tab:blue")
    line4, = ax[1].plot([], [], 'o', markersize=10, color="tab:red")

    def on_click(event):
        global toggle, labels
        if event.button is MouseButton.LEFT:
            if event.inaxes == button:
                labels[toggle].set_visible(False)
                toggle = not toggle
                labels[toggle].set_visible(True)
            if event.inaxes == ax[0]:
                x = event.xdata
                y = event.ydata
                if x>xrange[0] and x<xrange[1] and y>yrange[0] and y<yrange[1]:
                    if toggle:
                        equilibrium = fsolve(lambda x: f(0, x, pars), [x, y])
                        ax[0].scatter(equilibrium[0], equilibrium[1], s=70, color='tab:red', zorder=2)
                        point = [np.sin(equilibrium[0]), -np.cos(equilibrium[0])]
                        line3.set_data([0, point[0]], [0, point[1]])
                        line4.set_data(point[0], point[1])
                        fig.canvas.draw()
                    else:
                        sol = solve_ivp(pendulum, [0, T], [x, y], t_eval=np.linspace(0, T, Neval*Nframes),
                                        args=[pars], atol=1.e-8, rtol=1.e-6)
                        ani = animation.FuncAnimation(fig, update_graph, frames=range(1, Nframes),
                                        fargs=(sol.y, Nframes, Neval, ax[0], line1, line2, line3, line4),
                                        interval=50, repeat=False)
                        fig.canvas.draw()
 
    plt.connect('button_press_event', on_click)

def update_graph(k, u, Nframes, N, ax, line1, line2, line3, line4):
    soln = convert2circle(u[:, :k*N])
    line1.set_data(soln[0, :], soln[1, :])
    line2.set_data(soln[0, -1], soln[1, -1])
    point = [np.sin(soln[0, -1]), -np.cos(soln[0, -1])]
    line3.set_data([0, point[0]], [0, point[1]])
    line4.set_data(point[0], point[1])
    if k==Nframes-1:
        ax.plot(soln[0,:], soln[1,:], linewidth=2, color="tab:green", zorder=1)

def convert2circle(u):
    u[0,:] = np.mod(u[0,:]+np.pi, 2*np.pi) - np.pi
    jumps = abs(np.diff(u[0,:]))
    ind = 1+np.where(jumps > np.pi)[0]
    return np.insert(u, ind, np.NaN, axis=1)
        
def plotdf(rhs, pars, xrange, yrange, grid, ax):
    x = np.linspace(xrange[0], xrange[1], grid[0])
    y = np.linspace(yrange[0], yrange[1], grid[1])
    X, Y = np.meshgrid(x, y)
    DX, DY = rhs(0, [X, Y], pars)
    M = (np.hypot(DX, DY))
    M[M==0] = 1.0
    DX = DX/M
    DY = DY/M
    ax.quiver(X, Y, DX, DY, color='tab:gray', angles='xy', alpha=0.5)

We model a pendulum subjected to damping and a constant gravitational force using the second-order differential equation $$\frac{d^2\theta}{dt^2}+d\frac{d\theta}{dt}=-\sin\theta,$$ where $\theta$ is the angle between a vertical axis and the pendulum, and $d$ is the damping coefficient. Rewriting this equation as a first-order system, we obtain
$$
\begin{align}
\frac{d\theta}{dt} & = v \\
\frac{dv}{dt} & = - dv - \sin\theta,
\end{align}
$$
where $v$ is the angular velocity of the pendulum. Running the next cell will create the phase portrait of this model and an illustration of a physical pendulum. Clicking with the mouse in the phase portrait will simulate the pendulum motion starting with the initial condition selected by the mouse click.

In [4]:
def pendulum(t, u, damping):
    return [u[1], -damping*u[1] - np.sin(u[0])]

damping = 0.0
generate_phase_portrait(pendulum, damping, Nframes=150)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …